In [1]:
# Install correct dependencies
# Developed w/ Python 3.9.6

# %pip cache purge
# %pip install librosa==0.10.1
# %pip install llvmlite==0.43.0
# %pip install pandas==1.5.3
# %pip install protobuf==3.20.3
# %pip install click==8.1.3
# %pip install tensorflow-macos tensorflow-metal

from random_forest_utils import random_forest
from logistic_regression_utils import logistic_regression
import numpy as np
import pandas as pd
import librosa
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf

In [4]:
# Use Librose to perform Feature Extraction

DATA_DIR = "/Users/dmwelch/Development/NYU/MIR/finalProj/Data/genres_original"

def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=30)
    features = np.array([])

    # MFCC
    
    s = librosa.feature.mfcc(y=y, sr=sr)
    mfccs_mean = mfccs.mean(axis=1)
    features = np.hstack((features, mfccs_mean))

    # Chromagram
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = chroma.mean(axis=1)
    features = np.hstack((features, chroma_mean))

    # Spectral Contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean = contrast.mean(axis=1)
    features = np.hstack((features, contrast_mean))

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    tonnetz_mean = tonnetz.mean(axis=1)
    features = np.hstack((features, tonnetz_mean))

    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = zcr.mean()
    features = np.hstack((features, zcr_mean))

    # Spectral Roll-off
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = rolloff.mean()
    features = np.hstack((features, rolloff_mean))

    return features

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
# dataset = []

# for genre in genres:
#     genre_dir = os.path.join(DATA_DIR, genre)
#     for filename in os.listdir(genre_dir):
#         if filename.endswith('.wav'):
#             file_path = os.path.join(genre_dir, filename)
#             features = extract_features(file_path)
#             dataset.append([features, genre])


# # Convert to DataFrame
# df = pd.DataFrame(dataset, columns=['features', 'label'])

import pickle

# # Save the DataFrame
# with open('dataset.pkl', 'wb') as f:
#     pickle.dump(df, f)

# Load the DataFrame
with open('dataset.pkl', 'rb') as f:
    df = pickle.load(f)

X = np.array(df['features'].tolist())
y = np.array(df['label'].tolist())

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Portion 20% of data for testing, the rest for training
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [ ]:
# TRAINING THE MODELS, MAKING PREDICTIONS

# ADAboost
ada_clf = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)

# CATBoost
cat_clf = CatBoostClassifier(
    iterations=100,
    learning_rate=0.5,
    depth=6,
    loss_function='MultiClass',
    verbose=False,
    random_state=42
)
cat_clf.fit(X_train, y_train)
y_pred_cat = cat_clf.predict(X_test)

# Random Forest
rf = random_forest()
rf_model = rf.gen()
rf.fit(X_train, y_train)
rf_y_pred = rf.pred(X_test)

# 

In [ ]:
# GENERATING REPORTS

ada_report = classification_report(y_test, y_pred_ada, target_names=le.classes_, output_dict=True)
cat_report = classification_report(y_test, y_pred_cat, target_names=le.classes_, output_dict=True)
rf_report = classification_report(y_test, rf_y_pred, target_names=le.classes_, output_dict=True )
# rf_accuracy_score, rf_classification_report = rf.eval(y_test, rf_y_pred)
